In [ ]:
!kaggle datasets download sehriyarmemmedli/turboaz-cars-project
!unzip turboaz-cars-project.zip

#Data Cleaning

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Data Yükləmə
df = pd.read_csv('/content/cars.csv')
pd.set_option('display.max_columns', None)

# DataFrame-də ilkin yoxlama
df.head()
df.shape

# Sütun adlarının transformasiyası
df.columns = df.columns.str.strip().str.replace(r'\s+', '_', regex=True).str.lower()

# DataFrame-dəki dəyərlərin transformasiyası
df = df.map(lambda x: x.strip().replace(' ', '_').lower() if isinstance(x, str) else x)

# Dublikatları silirik
df = df.drop_duplicates(subset=['car_rel_url_x'], keep='last')

# Silinəcək sütunlar
silineceq_sutunlar_1 = ['id_x', 'car_rel_url_x', 'img_url', 'id_y', 'cars_id', 'car_rel_url_y',
                        'phone', 'car_details_id_x', 'car_rel_url', 'car_details_id_y', 'datetime_scrape',
                        'name', 'datetime_product', 'attributes', 'datetime', 'views', 'price_x',
                        'currency_x', 'production_year', 'city', 'yürüş', 'hour', 'day', 'updated',
                        'extra_info', 'vin', 'owner_name', 'shop_name', 'engine_displacement_unit',
                        'kilometrage_unit', 'marka','description', 'vəziyyəti']

# Sütunları silirik
df.drop(columns=silineceq_sutunlar_1, inplace=True)

# Numeric olan sütunları seçmək
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()

# 97%-dan böyük olan dəyərləri tapmaq
percentile_97 = np.percentile(df['kilometrage_num'], 97.)

# 97%-dan böyük olan dəyərləri tapıb 97%-ın qiymətinə bərabər edirik
df['kilometrage_num'] = np.where(df['kilometrage_num'] > percentile_97, percentile_97, df['kilometrage_num'])

# Buraxılış ili üzrə aşağı 1%-ı təyin edirik
percentile_1 = np.percentile(df['buraxılış_ili'], 1)
df['buraxılış_ili'] = np.where(df['buraxılış_ili'] < percentile_1, percentile_1, df['buraxılış_ili'])

# Şəhər sütununu təmizləmək
df['şəhər'] = df['şəhər'].apply(lambda x: 'bakı' if x == 'bakı' else 'digər')

# Standart olmayan ifadələri uyğun terminlərlə əvəz etmək üçün xəritə
standart_xerite = {
    'hetçbek 3 qapı': 'hatchback',
    'hetçbek 4 qapı': 'hatchback',
    'hetçbek 5 qapı': 'hatchback',
    'offroader suv 3 qapı': 'suv',
    'offroader suv 5 qapı': 'suv',
    'offroader suv açıq': 'suv',
    'pikap ikiqat kabin': 'pickup',
    'pikap tək kabin': 'pickup',
    'pikap bir yarım kabin': 'pickup',
    'universal 3 qapı': 'wagon',
    'universal 5 qapı': 'wagon',
    'suv kupe': 'suv coupe',
    'mikroavtobus': 'microbus',
    'mikrovan': 'microvan',
    'yük maşını': 'truck',
    'furqon': 'van',
    'karvan': 'caravan',
    'fayton': 'carriage',
    'qolfkar': 'golf cart',
    'spidster': 'speedster',
    'tarqa': 'targa'
}

# Təmizləmə funksiyası
def temizle_ban_novu(text):
    if pd.isna(text):
        return text
    text = text.lower().strip()
    text = re.sub(r'[_/,]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return standart_xerite.get(text, text)

# Ban növünü təmizləmək
df['ban_növü'] = df['ban_növü'].apply(temizle_ban_novu)

# Yeni sütunlar yaradaraq mühərrik gücünü çıxarmaq
df['güc'] = df['mühərrik'].str.extract(r'(\d+)_a\.g\.')
df['güc'] = pd.to_numeric(df['güc'], errors='coerce').fillna(0).astype(int)

# Yanacaq növü üçün yeni sütun
df['yanacaq_növü'] = df['mühərrik'].str.split('/').str[-1].str.replace('_', '')
df['yanacaq_növü'] = df['yanacaq_növü'].replace({
    'dizel': 'dizel',
    'benzin': 'benzin',
    'elektro': 'elektro',
    'hibrid': 'hibrid',
    'pluginhibrid': 'plug-in hibrid',
    'qaz': 'qaz',
    'hidrogen': 'hidrogen'
})

# 'mühərrik' sütununu silirik
df.drop(columns=['mühərrik'], inplace=True)

# Target dəyişənini hazırlayırıq (price_azn)
exchange_rates = {'usd': 1.7, 'eur': 1.67, 'azn': 1}
df['price_azn'] = df['price_y'] * df['currency_y'].map(exchange_rates)

# 'price_y' və 'currency_y' sütunlarını silirik
df.drop(columns=['price_y', 'currency_y'], inplace=True)

# Hazırkı ili alırıq və yaş hesablayırıq
current_year = datetime.datetime.now().year
df['yaş'] = current_year - df['buraxılış_ili']

df.drop(columns=['buraxılış_ili'], inplace=True)

# Numeric dəyişənləri yenidən yoxlayaq
df['annual_mileage'] = df['kilometrage_num'] / df['yaş']
df['power_to_displacement'] = df['güc'] / df['engine_displacement_num']

df['annual_mileage'].fillna(df['annual_mileage'].median(), inplace=True)
df['power_to_displacement'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['power_to_displacement'].fillna(df['power_to_displacement'].median(), inplace=True)

# Kategorik dəyişənləri təmizləyirik
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
df[categorical_cols] = df[categorical_cols].fillna('missing')

# Bəzi dəyişənləri daha sadə və optimallaşdırılmış formada saxlayırıq
top_6_colors = df['rəng'].value_counts().head(6).index
df['rəng'] = df['rəng'].apply(lambda x: x if x in top_6_colors else 'digər')

top_2_gearboxes = df['sürətlər_qutusu'].value_counts().head(2).index
df['sürətlər_qutusu'] = df['sürətlər_qutusu'].apply(lambda x: x if x in top_2_gearboxes else 'digər')


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

#Vizualizasiya

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Objekt tipli sütunları seçirik
object_columns = df.select_dtypes(include=['object']).columns

# 2-dən çox kateqoriyası olan sütunları filtrləyirik
for col in object_columns:
    if df[col].nunique() > 2:  # 2-dən çox unikal kateqoriya varsa
        # Kateqoriyaların paylanması
        category_counts = df[col].value_counts()

        # Bar plot ilə paylanmanı göstəririk
        plt.figure(figsize=(8, 6))
        sns.barplot(x=category_counts.index, y=category_counts.values, palette='viridis')
        plt.title(f"'{col}' sütunundakı kateqoriyaların paylanması", fontsize=14)
        plt.xlabel('Kateqoriyalar', fontsize=12)
        plt.ylabel('Say', fontsize=12)
        plt.xticks(rotation=45, ha='right')  # X oxunu döndəririk
        plt.show()


In [ ]:
# Kiçik bir alt dataset seçirik (10000 sətir)
sample_df = df.sample(n=50000, random_state=42)

#Lasso Model

MAE və R²-nin birgə istifadəsi:
Həm MAE (həqiqi səhvləri) həm də R² (modelin uyğunluğunu) izləyərək, modelin həm nəticə səhvlərini, həm də yaxşılığını qiymətləndirmək mümkündür. Birlikdə istifadə olunanda bu iki metrik:

MAE modelin nə qədər dəqiq olduğunu göstərir,
R² isə modelin məlumatlardakı dəyişkənliyi nə qədər yaxşı izah etdiyini göstərir.
Beləliklə, bu iki metrik həm modelin ümumi keyfiyyətini, həm də proqnozların nə qədər doğru olduğunu daha ətraflı şəkildə qiymətləndirməyə kömək edir

Niyə Lasso model secdim.

Təkrarlanan və ya əhəmiyyətsiz xüsusiyyətləri kənara qoymaq: Lasso modelində, cərimə tətbiq edildikcə, əhəmiyyətsiz xüsusiyyətlər sıfırlanır. Bu, xüsusiyyətlərin səmərəli seçilməsini və modelin sadələşdirilməsini təmin edir.

Daha az overfitting riski: Lasso'nun cərimə tətbiq etməsi, overfitting (çox yaxşı təlim məlumatlarına uyğunlaşma) riskini azaldır. Bu, xüsusən çox sayda xüsusiyyətə sahib datasetlərdə vacibdir.


In [ ]:
# Lazım olan kitabxanaları idxal edirik
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from scipy.stats import uniform


# X və y-nin ayrılması
X = sample_df.drop(columns='price_azn')  # Target variable
y = sample_df['price_azn']  # Target variable

# Verilənləri train və test hissələrinə ayırırıq
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kateqorial və sayısal dəyişənləri ayırırıq
numerical_cols = ['engine_displacement_num', 'kilometrage_num', 'güc', 'yaş', 'annual_mileage', 'power_to_displacement']
categorical_cols = ['barter', 'loan', 'salon', 'spare_parts', 'vip', 'featured', 'ban_növü',
                    'hansı_bazar_üçün_yığılıb', 'model', 'qəzalı', 'rəng', 'sahiblər', 'sürətlər_qutusu',
                    'yeni', 'yerlərin_sayı', 'ötürücü', 'şəhər', 'yanacaq_növü']

# Sayısal və kateqorial verilənlər üçün fərqli transformasiya addımları
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Missing dəyərləri orta ilə əvəz edirik
    ('scaler', StandardScaler()),  # Miqyaslama
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Polynomial Features
    ('pca', PCA(n_components=5))  # PCA ilə ölçü azaldılması
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Missing dəyərləri ən çox təkrarlanan dəyərlə əvəz edirik
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encoding
])

# Hər iki tip dəyişən üçün transformasiyaları birləşdiririk
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Model və preprocessing addımlarını birləşdiririk
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(random_state=42))  # Lasso regression modelini istifadə edirik
])

# Modeli fit edirik
model_pipeline.fit(X_train, y_train)

# Test datası ilə modelin qiymətləndirilməsi
y_pred = model_pipeline.predict(X_test)

# Performans metrikləri
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R²: {r2}")
print(f"RMSE: {rmse}")

# Kross-validasiya ilə daha yaxşı qiymətləndirmə
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model_pipeline, X, y, cv=kf, scoring='neg_mean_squared_error')

print(f"5-fold CV Mean RMSE: {np.sqrt(-cv_scores.mean())}")


# Hyperparameter tuning (RandomizedSearchCV ilə)
param_dist = {
    'regressor__alpha': uniform(0.01, 10),  # Alpha parametri 0.01 ilə 10 arasında for Lasso
    'regressor__max_iter': [1000, 2000, 5000, 10000],  # Iterasiya sayını dəyişirik for Lasso
}

# RandomizedSearchCV ilə hyperparameter tuning
random_search = RandomizedSearchCV(Lasso, param_distributions=param_dist, n_iter=10, cv=5,
                                   n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
random_search = RandomizedSearchCV(model_pipeline, param_distributions=param_dist, n_iter=10, cv=kf,
                                   n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

print(f"Best Parameters: {random_search.best_params_}")
print(f"Best CV RMSE: {np.sqrt(-random_search.best_score_)}")


#RandomForestRegressor

RandomForestRegressor niyə seçdim.

Yüksək performanslı və çox yönlüdür: Random Forest, çox sayda qərar ağacından ibarət bir ensemble (topluluk) modelidir və ümumilikdə yaxşı nəticələr verir. Bu model həm sayısal həm də kateqorial verilənlər üzərində yüksək performans göstərə bilir.

In [ ]:
 # Lazım olan kitabxanaları idxal edirik
from sklearn.ensemble import RandomForestRegressor

# X və y-nin ayrılması
X = sample_df.drop(columns='price_azn')  # Target variable
y = sample_df['price_azn']  # Target variable

# Verilənləri train və test hissələrinə ayırırıq
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kateqorial və sayısal dəyişənləri ayırırıq
numerical_cols = ['engine_displacement_num', 'kilometrage_num', 'güc', 'yaş', 'annual_mileage', 'power_to_displacement']
categorical_cols = ['barter', 'loan', 'salon', 'spare_parts', 'vip', 'featured', 'ban_növü',
                    'hansı_bazar_üçün_yığılıb', 'model', 'qəzalı', 'rəng', 'sahiblər', 'sürətlər_qutusu',
                    'yeni', 'yerlərin_sayı', 'ötürücü', 'şəhər', 'yanacaq_növü']

# Sayısal və kateqorial verilənlər üçün fərqli transformasiya addımları
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Missing dəyərləri orta ilə əvəz edirik
    ('scaler', StandardScaler()),  # Miqyaslama
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Polynomial Features
    ('pca', PCA(n_components=5))  # PCA ilə ölçü azaldılması
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Missing dəyərləri ən çox təkrarlanan dəyərlə əvəz edirik
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encoding
])

# Hər iki tip dəyişən üçün transformasiyaları birləşdiririk
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Model və preprocessing addımlarını birləşdiririk
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))  # Random Forest Regressor istifadə edirik
])

# Modeli fit edirik
model_pipeline.fit(X_train, y_train)

# Test datası ilə modelin qiymətləndirilməsi
y_pred = model_pipeline.predict(X_test)

# Performans metrikləri
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R²: {r2}")
print(f"RMSE: {rmse}")

# Kross-validasiya ilə daha yaxşı qiymətləndirmə
cv_scores = cross_val_score(model_pipeline, X, y, cv=3, scoring='neg_mean_squared_error')

print(f"3-fold CV Mean RMSE: {np.sqrt(-cv_scores.mean())}")


# Hyperparameter tuning (RandomizedSearchCV ilə)
param_dist = {
    'regressor__n_estimators': [100, 200, 500],  # Random Forest üçün ağac sayını dəyişirik
    'regressor__max_depth': [None, 10, 20, 30],  # Random Forest ağaclarının maksimum dərinliyini dəyişirik
    'regressor__min_samples_split': [2, 5, 10],  # Ağacın bölünməsini minimum nümunə sayını dəyişirik
    'regressor__min_samples_leaf': [1, 2, 4],  # Ağacın yarpağında minimum nümunə sayını dəyişirik
}

random_search = RandomizedSearchCV(model_pipeline, param_distributions=param_dist, n_iter=10, cv=3,
                                   n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

print(f"Best Parameters: {random_search.best_params_}")
print(f"Best CV RMSE: {np.sqrt(-random_search.best_score_)}")


R²: 0.728, modelin təxminlərinin hədəf dəyişkənlə nə qədər yaxşı uyğunlaşdığını göstərir. Əvvəlki nəticədə bu dəyər 0.36 idi, yəni əvvəlki model çox zəif təxminlər vermişdi.
RMSE: 13527.22, modelin səhvlərinin ölçüsünü göstərir. Bu da əvvəlki 20745.60-dan daha aşağıdır, yəni modelin səhvləri kiçilib.
CV Mean RMSE: 13711.80, 3-fold cross-validation ilə əldə edilən orta RMSE qiymətidir. Bu da modelin ümumi performansını yaxşılaşdırmağa kömək edən bir metrikdir.